## RLDMUU 2025
#### Backward Induction
jakub.tluczek@unine.ch

Today your task would be to implement the backwards induction algorithm for the following MDP (you can also find it in `src/MDP/MDP.py` on our github):

In [4]:
import numpy as np

## This a discrete MDP with a finite number of states and actions
class DiscreteMDP:
    ## initalise a random MDP with
    ## n_states: the number of states
    ## n_actions: the number of actions
    ## Optional arguments:
    ## P: the state-action-state transition matrix so that P[s,a,s_next] is the probability of s_next given the current state-action pair (s,a)
    ## R: The state-action reward matrix so that R[s,a] is the reward for taking action a in state s.
    def __init__(self, n_states, n_actions, P = None, R = None):
        self.n_states = n_states # the number of states of the MDP
        self.n_actions = n_actions # the number of actions of the MDP
        if (P is None):
            self.P = np.zeros([n_states, n_actions, n_states]) # the transition probability matrix of the MDP so that P[s,a,s'] is the probabiltiy of going to s' from (s,a)
            for s in range(self.n_states):
                for a in range(self.n_actions):
                    self.P[s,a] = np.random.dirichlet(np.ones(n_states)) # generalisation of Beta to multiple outcome
        else:
            self.P = P
        if (R is None):
            self.R = np.zeros([n_states, n_actions]) # the expected reward for each action and state
            # generate uniformly random transitions and 0.1 bernoulli rewards
            for s in range(self.n_states):
                for a in range(self.n_actions):
                    self.R[s,a] = np.round(np.random.uniform(), decimals=1)
        else:
            self.R = R
        
        # check transitions
        for s in range(self.n_states):
            for a in range(self.n_actions):
                #print(s,a, ":", self.P[s,a,:])
                assert(abs(np.sum(self.P[s,a,:])-1) <= 1e-3)
                assert((self.P[s,a,:] <= 1).all())
                assert((self.P[s,a,:] >= 0).all())
                
    # get the probability of next state j given current state s, action a, i.e. P(j|s,a)
    def get_transition_probability(self, state, action, next_state):
        return self.P[state, action, next_state]
    
    # get the vector of probabilities over next states P( . | s,a)
    def get_transition_probabilities(self, state, action):
        return self.P[state, action]
    
    # Get the reward for the current state action.
    # It can also be interpreted as the expected reward for the state and action.
    def get_reward(self, state, action):
        return self.R[state, action]

### Backward induction

As a reminder, in the backward induction algorithm we consider an MDP with finite horizon $T$, and for each step of the algorithm, we compute:

$$ V_t(s) = max_{a \in A} \left[ R(s,a) + \sum_{s' \in S} P(s' | s,a)V_{t+1}(s') \right]$$

where $R(s,a)$ is a reward received by picking action $a$ in state $s$, $P(s'|s,a)$ is the probability of transitioning into next state $s'$, and $V_{t+1}(s')$ is the value of said next state at time $t+1$. We can also say, that for the last timestep (with index $T-1$) the next state value is 0 for every state $V_T(s) = 0$. Consecutively, the action $a$ which maximizes $V_t(s)$ can be described as policy $\pi_t(s)$ at state $s$ and time $t$, 

Your task is to implement this algorithm. Remember to do the inverse iteration, and iterate from $T-1$ to $0$, not the other way around. Your function should return matrix of state values for each $s$ and $t$, as well as resulting policy:

In [5]:
# TODO: Implement backwards induction
def backwards_induction(mdp, T):
    n_states = mdp.n_states
    n_actions = mdp.n_actions
    P = mdp.P
    R = mdp.R
    V = np.zeros((n_states, T + 1))
    pi = np.zeros((n_states, T), dtype=int)
    for t in reversed(range(T)):
        for s in range(n_states):
            term_vec = np.zeros(n_actions)
            for a in range(n_actions):
                # Accumulate over all next states
                for s_prime in range(n_states):
                    term_vec[a] += P[s, a, s_prime] * (R[s, a] + V[s_prime, t + 1])
            V[s, t] = np.max(term_vec)
            pi[s, t] = np.argmax(term_vec)
    return V, pi
        

In [3]:
STATES = 5
ACTIONS = 3

T = 15

mdp = DiscreteMDP(STATES, ACTIONS)

V, policy = backwards_induction(mdp, T)

print(V)

print(policy)


[[10.69851051  9.95410247  9.20969443  8.46528638  7.72087842  6.9764701
   6.23206211  5.48765799  4.74322939  3.99885962  3.25456555  2.50895786
   1.76843901  1.02256485  0.2         0.        ]
 [11.26008878 10.51568074  9.7712727   9.02686466  8.28245663  7.53804856
   6.79364061  6.0492331   5.30482302  4.56042565  3.81603539  3.07155295
   2.32801241  1.58477604  0.8         0.        ]
 [11.1706267  10.42621866  9.68181062  8.93740258  8.19299455  7.44858655
   6.7041782   5.9597713   5.21536347  4.47094213  3.72661916  2.98209955
   2.23742477  1.49703907  0.8         0.        ]
 [11.31757258 10.57316454  9.8287565   9.08434846  8.33994039  7.5955325
   6.85112412  6.10671546  5.3623152   4.61787654  3.87349135  3.12940003
   2.38291109  1.64317255  0.9         0.        ]
 [11.20041515 10.45600711  9.71159907  8.96719103  8.22278298  7.47837494
   6.73396712  5.98955795  5.24515172  4.50075047  3.75627454  3.0120649
   2.26758388  1.51968656  0.8         0.        ]]
[[1 1 1